# ELYZA-tasks-100 による性能評価

In [ ]:
!pip install accelerate datasets jsonlines

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.5 MB/s eta 0:00:00


## ELYZA-tasks-100 データセットのダウンロード

モデルのダウンロードには 🤗datasets ライブラリを用いる。

データは `/content/dataset.jsonl` に保存する。

In [ ]:
import json
from pathlib import Path

from datasets import load_dataset

# Load the dataset
ds = load_dataset("elyza/ELYZA-tasks-100")

# Function to convert dataset to JSONL format and print
def dataset_to_jsonl(dataset, filename):
    with open(filename, 'w', encoding='utf-8') as file:
        for entry in dataset:
            # Construct JSON object
            json_obj = {
                "input_text": entry['input'],
                "output_text": entry['output'],
                "eval_aspect": entry['eval_aspect']
            }

            # Write JSON object to file in JSONL format
            json_str = json.dumps(json_obj, ensure_ascii=False)
            file.write(json_str + '\n')

# Convert and write the dataset to a file in JSONL format
path_jsonl = Path("/content/dataset.jsonl")

if (path_jsonl.parent is not None) and (not path_jsonl.parent.exists()):
    path_jsonl.parent.mkdir(parents=True, exist_ok=True)

dataset_to_jsonl(ds["test"], path_jsonl)


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating test split: 0 examples [00:00, ? examples/s]

## 評価

ELYZA-tasks-100 データセットを用いて、指定したモデルの評価を実行する。

### ハイパーパラメータについて

ハイパーパラメータは 🤗HuggingFace の Code Snippet のとおりとする（航海者が指定したハイパーパラメータを、チューン済みとみなす）。
ただし、長文の回答を要求する設問もあるため、`max_new_tokens` は `1024` に統一する。

In [ ]:
import json
import jsonlines
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Setup model and tokenizer
model_name = "stabilityai/japanese-stablelm-instruct-beta-7b"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, low_cpu_mem_usage=True, device_map="auto")

def format_prompt(input_text):
    prompt_template = """<s>[INST] <<SYS>>\nあなたは役立つアシスタントです。\n<<SYS>>\n\nユーザの質問に答えてください。\n\n{input}[/INST]"""
    return prompt_template.format(input=input_text)

def generate_text(input_text):
    formatted_prompt = format_prompt(input_text)
    input_ids = tokenizer.encode(
        formatted_prompt,
        add_special_tokens=False,
        return_tensors="pt"
    )

    # Set seed for reproducibility
    seed = 42
    torch.manual_seed(seed)

    tokens = model.generate(
        input_ids.to(device=model.device),
        max_new_tokens=1024,
        temperature=0.99,
        top_p=0.95,
        do_sample=True,
    )

    # Remove the input tokens from the generated tokens before decoding
    output_tokens = tokens[0][len(input_ids[0]):]
    return tokenizer.decode(output_tokens, skip_special_tokens=True)

def process_dataset(input_file, output_file):
    # Read dataset and generate predictions
    with jsonlines.open(input_file) as reader, jsonlines.open(output_file, mode='w') as writer:
        for obj in reader:
            prompt = obj['input_text']
            generated_text = generate_text(prompt)
            print(generated_text)
            writer.write({"pred": generated_text})

# Process the dataset
input_dataset = '/content/dataset.jsonl'
output_predictions = '/content/preds.jsonl'
process_dataset(input_dataset, output_predictions)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

 やる気を失ったときにあなたを刺激するためにできることはたくさんあります。
 - あなたが愛する活動に時間を過ごす
 - あなたが読みたい本を読む
 - その日の前にいくつかの小さな課題を考える
 - あなたが信じるものであなた自身を励ます
- あなたが持つ仕事の理由に焦点を合わせ、納得していると感じる

これらのアイデアのいくつかは効果的であり、それらを使用して熱意を取り戻すのに役立つかもしれません。しかし、彼らはあなたが熱意を失っているときに、あなたがそれらを見つけたり、あなたを助けるのに役立ちます。
 - 日光を吸収し、それをあなたの運転を減らす
 - 単語の意味を検索したり、別の職業を見つけるためのヒントを見つけたりします
 - 熱意とあなたを助けるときにあなたを助ける職場の人々を探してください
 - 新しい方法を見つけるのを待ってください、またはあなたの日付の中で
 - 仕事から離れてリラックスするために時間を割く
 
最も重要なことは、あなたがあなたがあなたの人生の重要なものであると感じるものと一致し、納得すると感じる人々の助けを借りて熱意を保つことです。
ああ、あなたは面白いね。私があなたがそれを行うのを助けることができるなら、考えてみましょう...
 オリンパス山脈、ギリシャの首都、神々、アテネ、デロス諸島、神話など、学ぶべきことがたくさんあります。
 あなたの説明のように、ゴミ圧縮機はゴミを一網打尽にして、費用の軽減と環境問題を解決することができますね。
 オリオンの彼方に、デイビッド・ブリーデン著、

踊る大脳、クリスチャン・スコット・スタブリッツ著、

カリオンの襲撃、アリソン・マンチェスター著、

コンセクライス、スティーブン・ブライクマイアー著、

アンダー・ライフ、ピーター・ワトスン著、

ドラゴンの騎士団、マイクル・クリップス著、

雌猫歳過、リサ・ゴールドバーグ著、

催眠術士の歴史、ジョン・バーディナス著、

犬の彼方に、スザンナ・クロンカイト著、

オールド・シティ、ジャクリーン・デイ・ブラッドベリ著、

リトル・システム、メイアン・カニングハム著。

「犬の彼方に」は、ラブコメSFとなる。

「アリソン・マンチェスター」の「カリオンの襲撃」は、スパイ小説となる。

「スティーブン・ブライクマイヤー」の「コンセクライス」は、科学フ